<a href="https://colab.research.google.com/github/mariahelenass/MLflow/blob/main/MLflow_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143860 sha256=e564fbf7b07f0aedab9ef4f3be99fb9bbfc5a809b9dc3c562f2ccc8d83be61fb
  Stored in directory: /root/.cache/pip/wheels/a

In [ ]:
import mlflow
import mlflow.sklearn

In [ ]:
df = pd.read_csv('/content/Credit.csv')

In [ ]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


In [ ]:
for col in df.columns:
  if df[col].dtype == 'object':
    df[col] = df[col].astype('category').cat.codes

In [ ]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,2,6,1,7,1169,0,3,4,3,2,...,2,67,1,1,2,3,1,1,1,1
1,1,48,3,7,5951,3,0,2,0,2,...,2,22,1,1,1,3,1,0,1,0
2,0,12,1,4,2096,3,1,2,3,2,...,2,49,1,1,1,2,2,0,1,1
3,2,42,3,5,7882,3,1,2,3,1,...,0,45,1,0,1,3,2,0,1,1
4,2,24,2,1,4870,3,0,3,3,2,...,1,53,1,0,2,3,2,0,1,0


In [ ]:
previsores = df.iloc[:,0:20].values
classe = df.iloc[:,20].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(previsores, classe, test_size=0.3, random_state=123)

In [ ]:
def treina_modelo(n_estimators):

  mlflow.set_experiment("rfexperimento")
  with mlflow.start_run():
    random_forest = RandomForestClassifier(n_estimators)
    random_forest.fit(X_train, y_train)
    predict = random_forest.predict(X_test)

    # registrando os parametros
    mlflow.log_params({"n_estimators": n_estimators})

    # métricas
    accuracy = accuracy_score(y_test, predict)
    recall = recall_score(y_test, predict)
    precision = precision_score(y_test, predict)
    f1 = f1_score(y_test, predict)
    auc = roc_auc_score(y_test, predict)
    log = log_loss(y_test, predict)

    # registrando métricas
    mlflow.log_metric("acuracia", accuracy)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("auc", auc)
    mlflow.log_metric("log", log)

    # gráficos
    confusion = confusion_matrix(y_test, predict)
    plt.savefig("confusion_rf.png")

    # registrando gráficos
    mlflow.log_artifact("confusion_rf.png")

    # modelo
    mlflow.sklearn.log_model(random_forest, "Modelo Random Forest")

    # informações da execução:
    print("Modelo: ", mlflow.active_run().info.run_uuid)

  mlflow.end_run()

In [ ]:
arvores = [50, 100, 500, 750, 1000]

for i in arvores:
  treina_modelo(i)

Modelo:  226aafa0b966492ca71109fd1bdea183
Modelo:  783329d0abaa4a819a82227493b04e90
Modelo:  09ff63c4ddb148f98cc4e28ef166cd1a
Modelo:  2b9e8105e2df470faa4c8803616c31e8
Modelo:  b1d597c534ec4f69906a23b673ad6f48


<Figure size 640x480 with 0 Axes>